In [1]:
import torch 
# import dataset from torch geometric
from torch_geometric.data import Data, DataLoader, HeteroData
from tqdm.auto import tqdm 
import os 
from pathlib import Path
from copy import deepcopy

/opt/conda/envs/pyg/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
for file in tqdm(os.listdir("v2_directed_withdegreecount_heterogeneous")):
    # skip if exiss already
    # if Path("v2_undirected_withdegreecount/" + file).is_file() and Path("v2_undirected_withdegreecount_heterogeneous/" + file).is_file():
        # continue
    
    data = torch.load("v2_directed_withdegreecount_heterogeneous/" + file)
    if data is False:
        continue
    
    
    
    TOTAL_undirected = torch.cat([data['node','TOTAL','node'].edge_index, data['node','TOTAL','node'].edge_index.flip(0)], dim=1)
    # remove duplicates
    TOTAL_undirected = torch.unique(TOTAL_undirected, dim=1, sorted=True)
    
    new_data = Data(
        x = data['node'].x,
        edge_index = TOTAL_undirected,
        y = data['node'].y
    )
    
    
    new_heterogeneous_data = deepcopy(data)
    for edge_type in data.edge_types:
        # if there is non unique, print file
        # if not torch.equal(data[edge_type].edge_index, torch.unique(data[edge_type].edge_index, dim=1, sorted=True)):
        #     print(file)
        #     print(edge_type)
        
        ud = data[edge_type].edge_index
        ud = torch.cat([ud, ud.flip(0)], dim=1)
        # remove duplicates
        ud = torch.unique(ud, dim=1, sorted=True)
        new_heterogeneous_data[edge_type].edge_index = ud
        
    
    
    # save to v2_directed_withdegreecount
    torch.save(new_data, "v2_undirected_withdegreecount/" + file)
    torch.save(new_data, "v2_undirected_withdegreecount_heterogeneous/" + file)
    

  2%|▏         | 5447/329783 [00:01<01:55, 2809.92it/s] 


KeyboardInterrupt: 

In [41]:
new_heterogeneous_data

HeteroData(
  node={
    num_nodes=32,
    x=[32, 162],
    y=[1],
  },
  (node, TOTAL, node)={ edge_index=[2, 74] },
  (node, AST, node)={ edge_index=[2, 32] },
  (node, CFG, node)={ edge_index=[2, 22] },
  (node, CG, node)={ edge_index=[2, 2] },
  (node, DFG, node)={ edge_index=[2, 23] }
)

In [37]:
data.edge_types

[('node', 'TOTAL', 'node'),
 ('node', 'AST', 'node'),
 ('node', 'CFG', 'node'),
 ('node', 'CG', 'node'),
 ('node', 'DFG', 'node')]

In [14]:
e = torch.tensor([[2,3,11,4,5,11,6,2],[4,3,11,4,2,11,4,4]])

In [25]:
data['node','TOTAL','node'].edge_index.shape

torch.Size([2, 20])

In [22]:
TOTAL_undirected.shapew

torch.Size([2, 36])

In [28]:
TOTAL_undirected

tensor([[ 0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,  4,
          4,  5,  5,  5,  5,  6,  6,  6,  7,  7,  8,  8,  8,  8,  9,  9, 10, 10],
        [ 1,  8,  0,  2,  4,  6,  7,  1,  3,  6,  2,  4,  5,  6,  8,  1,  3,  5,
          9,  3,  4,  7, 10,  1,  2,  3,  1,  5,  0,  3,  9, 10,  4,  8,  5,  8]],
       device='cuda:0')

In [17]:
torch.unique(e, dim=1, sorted=True)

tensor([[ 2,  3,  4,  5,  6, 11],
        [ 4,  3,  4,  2,  4, 11]])